In [1]:
import dash
from dash import dcc, html, dash_table, Input, Output, State, callback
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import requests
from io import StringIO
import warnings
import json
warnings.filterwarnings('ignore')

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Fantasy Football Draft Dashboard"

def get_draft_recommendation(available_df, current_round, drafted_players):
    """Get draft recommendations using Z-Score analysis"""
    
    if len(available_df) == 0:
        return html.Div(" No players available!")
    
    # Always use Z-Score for recommendations (better value identification)
    best_player = available_df.nlargest(1, 'ZScore').iloc[0]
    
    # Analyze current roster
    drafted_positions = []
    for player_name in drafted_players:
        player_data = df_global[df_global['Player'] == player_name]
        if len(player_data) > 0:
            drafted_positions.append(player_data.iloc[0]['Pos'])
    
    # Early rounds logic (1-3) - Focus on elite Z-scores
    if current_round <= 3:
        elite_players = available_df[available_df['ZScore'] >= 1.5].nlargest(3, 'ZScore')
        if len(elite_players) > 0:
            top_pick = elite_players.iloc[0]
            tier_desc = f"Elite {top_pick['Pos']} (+{top_pick['ZScore']:.1f} vs avg)"
            return html.Div([
                html.H4(f" ROUND {current_round} RECOMMENDATION", 
                       style={'color': '#28a745', 'marginBottom': '10px'}),
                html.H2(f" DRAFT: {top_pick['Player']}", 
                       style={'color': '#1f77b4', 'marginBottom': '5px'}),
                html.P(f"{top_pick['Pos']} • {top_pick['Team']} • {top_pick['FantasyPoints']:.1f} fantasy points",
                      style={'fontSize': '16px', 'marginBottom': '5px'}),
                html.P(tier_desc, 
                      style={'fontWeight': 'bold', 'color': '#ff9800', 'marginBottom': '10px'}),
                html.P("Elite value - significant advantage over position!", 
                      style={'fontStyle': 'italic', 'color': '#666'})
            ])
    
    # Mid rounds - Position needs with Z-score consideration
    elif current_round <= 8:
        needs_qb = drafted_positions.count('QB') == 0
        needs_te = drafted_positions.count('TE') == 0
        needs_rb = drafted_positions.count('RB') < 2
        needs_wr = drafted_positions.count('WR') < 2
        
        if needs_qb:
            top_qb = available_df[available_df['Pos'] == 'QB'].nlargest(1, 'ZScore')
            if len(top_qb) > 0:
                qb = top_qb.iloc[0]
                value_desc = f"Z-Score: +{qb['ZScore']:.1f} ({qb['ValueTier']})"
                return html.Div([
                    html.H4(f" ROUND {current_round} RECOMMENDATION", 
                           style={'color': '#28a745', 'marginBottom': '10px'}),
                    html.H2(f"QB NEEDED: {qb['Player']}", 
                           style={'color': '#ff9800', 'marginBottom': '5px'}),
                    html.P(f"QB • {qb['Team']} • {qb['FantasyPoints']:.1f} fantasy points",
                          style={'fontSize': '16px', 'marginBottom': '5px'}),
                    html.P(value_desc,
                          style={'fontWeight': 'bold', 'color': '#1f77b4', 'marginBottom': '10px'}),
                    html.P("Fill your QB need with solid value!", 
                          style={'fontStyle': 'italic', 'color': '#666'})
                ])
        
        if needs_te:
            top_te = available_df[available_df['Pos'] == 'TE'].nlargest(1, 'ZScore')
            if len(top_te) > 0:
                te = top_te.iloc[0]
                value_desc = f"Z-Score: +{te['ZScore']:.1f} ({te['ValueTier']})"
                return html.Div([
                    html.H4(f"ROUND {current_round} RECOMMENDATION", 
                           style={'color': '#28a745', 'marginBottom': '10px'}),
                    html.H2(f"TE NEEDED: {te['Player']}", 
                           style={'color': '#ff9800', 'marginBottom': '5px'}),
                    html.P(f"TE • {te['Team']} • {te['FantasyPoints']:.1f} fantasy points",
                          style={'fontSize': '16px', 'marginBottom': '5px'}),
                    html.P(value_desc,
                          style={'fontWeight': 'bold', 'color': '#1f77b4', 'marginBottom': '10px'}),
                    html.P("Secure your TE position!", 
                          style={'fontStyle': 'italic', 'color': '#666'})
                ])
    
    # Default recommendation - best Z-score available
    value_desc = f"Z-Score: +{best_player['ZScore']:.1f} ({best_player['ValueTier']})"
    return html.Div([
        html.H4(f"ROUND {current_round} RECOMMENDATION", 
               style={'color': '#28a745', 'marginBottom': '10px'}),
        html.H2(f"BEST VALUE: {best_player['Player']}", 
               style={'color': '#1f77b4', 'marginBottom': '5px'}),
        html.P(f"{best_player['Pos']} • {best_player['Team']} • {best_player['FantasyPoints']:.1f} fantasy points",
              style={'fontSize': '16px', 'marginBottom': '5px'}),
        html.P(value_desc,
              style={'fontWeight': 'bold', 'color': '#1f77b4', 'marginBottom': '10px'}),
        html.P("Best position-relative value available", 
              style={'fontStyle': 'italic', 'color': '#666'})
    ])

def create_enhanced_sample_data():
    """Create a comprehensive sample dataset with 100+ players"""
    
    # Extended player database with 2024 season stats
    enhanced_data = {
        'Player': [
            # QBs (Top 15)
            'Josh Allen', 'Lamar Jackson', 'Dak Prescott', 'Tua Tagovailoa', 'Brock Purdy',
            'Jalen Hurts', 'Josh Jacobs', 'Joe Burrow', 'Aaron Rodgers', 'Patrick Mahomes',
            'Russell Wilson', 'Kyler Murray', 'Geno Smith', 'Derek Carr', 'Baker Mayfield',
            
            # RBs (Top 25)
            'Christian McCaffrey', 'Derrick Henry', 'Saquon Barkley', 'Kyren Williams', 'Josh Jacobs',
            'Alvin Kamara', 'Rachaad White', 'Jonathan Taylor', 'Kenneth Walker III', 'Tony Pollard',
            'Bijan Robinson', 'Breece Hall', 'Isiah Pacheco', 'David Montgomery', 'Javonte Williams',
            'Aaron Jones', 'Miles Sanders', 'Najee Harris', 'Joe Mixon', 'Ezekiel Elliott',
            'James Cook', 'Rhamondre Stevenson', 'Travis Etienne', 'Dameon Pierce', 'Alexander Mattison',
            
            # WRs (Top 30)
            'CeeDee Lamb', 'Tyreek Hill', 'Amon-Ra St. Brown', 'Puka Nacua', 'Mike Evans',
            'Stefon Diggs', 'A.J. Brown', 'Davante Adams', 'DK Metcalf', 'Chris Olave',
            'Keenan Allen', 'DeVonta Smith', 'Garrett Wilson', 'Cooper Kupp', 'Michael Pittman Jr.',
            'Ja\'Marr Chase', 'Calvin Ridley', 'DJ Moore', 'Tee Higgins', 'Jaylen Waddle',
            'Tank Dell', 'Zay Flowers', 'Chris Godwin', 'Brandon Aiyuk', 'Diontae Johnson',
            'Terry McLaurin', 'Courtland Sutton', 'George Pickens', 'Jordan Addison', 'Rashee Rice',
            
            # TEs (Top 15)
            'Travis Kelce', 'Trey McBride', 'George Kittle', 'Sam LaPorta', 'Evan Engram',
            'Mark Andrews', 'T.J. Hockenson', 'Kyle Pitts', 'David Njoku', 'Dallas Goedert',
            'Pat Freiermuth', 'Jake Ferguson', 'Dalton Kincaid', 'Tyler Higbee', 'Hunter Henry',
            
            # Kickers (Top 10)
            'Justin Tucker', 'Harrison Butker', 'Tyler Bass', 'Jake Moody', 'Brandon McManus',
            'Daniel Carlson', 'Matt Gay', 'Younghoe Koo', 'Dustin Hopkins', 'Jake Elliott',
            
            # Defenses (Top 10)
            'Pittsburgh', 'Cleveland', 'Baltimore', 'Buffalo', 'San Francisco',
            'Dallas', 'Philadelphia', 'New York Jets', 'Miami', 'Kansas City'
        ],
        'Team': [
            # QBs
            'BUF', 'BAL', 'DAL', 'MIA', 'SF', 'PHI', 'LV', 'CIN', 'NYJ', 'KC',
            'DEN', 'ARI', 'SEA', 'NO', 'TB',
            # RBs  
            'SF', 'BAL', 'NYG', 'LAR', 'LV', 'NO', 'TB', 'IND', 'SEA', 'DAL',
            'ATL', 'NYJ', 'KC', 'DET', 'DEN', 'GB', 'CAR', 'PIT', 'HOU', 'NE',
            'BUF', 'NE', 'JAX', 'HOU', 'MIN',
            # WRs
            'DAL', 'MIA', 'DET', 'LAR', 'TB', 'HOU', 'PHI', 'LV', 'SEA', 'NO',
            'LAC', 'PHI', 'NYJ', 'LAR', 'IND', 'CIN', 'TEN', 'CHI', 'CIN', 'MIA',
            'HOU', 'BAL', 'TB', 'SF', 'PIT', 'WAS', 'DEN', 'PIT', 'MIN', 'KC',
            # TEs
            'KC', 'ARI', 'SF', 'DET', 'JAX', 'BAL', 'MIN', 'ATL', 'CLE', 'PHI',
            'PIT', 'DAL', 'BUF', 'LAR', 'NE',
            # Kickers
            'BAL', 'KC', 'BUF', 'SF', 'JAX', 'LV', 'LAR', 'ATL', 'CLE', 'PHI',
            # Defenses
            'PIT', 'CLE', 'BAL', 'BUF', 'SF', 'DAL', 'PHI', 'NYJ', 'MIA', 'KC'
        ],
        'Pos': (
            ['QB'] * 15 + ['RB'] * 25 + ['WR'] * 30 + ['TE'] * 15 + 
            ['K'] * 10 + ['DEF'] * 10
        )
    }
    
    # Generate realistic fantasy stats based on position
    num_players = len(enhanced_data['Player'])
    
    # Initialize all stats to 0
    for stat in ['PassYd', 'PassTD', 'Int', 'RushYd', 'RushTD', 'Rec', 'RecYd', 'RecTD', 'FumLost']:
        enhanced_data[stat] = [0] * num_players
    
    # Add realistic stats by position
    np.random.seed(42)  # For consistent results
    for i, pos in enumerate(enhanced_data['Pos']):
        if pos == 'QB':
            # QB stats - declining from elite to average
            rank = i  # 0-14 for QBs
            enhanced_data['PassYd'][i] = int(4500 - (rank * 200) + np.random.randint(-200, 200))
            enhanced_data['PassTD'][i] = int(35 - (rank * 1.5) + np.random.randint(-3, 3))
            enhanced_data['Int'][i] = int(8 + (rank * 0.5) + np.random.randint(-2, 4))
            enhanced_data['RushYd'][i] = int(max(0, 400 - (rank * 20) + np.random.randint(-100, 100)))
            enhanced_data['RushTD'][i] = int(max(0, 8 - (rank * 0.4) + np.random.randint(-2, 2)))
            
        elif pos == 'RB':
            # RB stats
            rank = i - 15  # 0-24 for RBs
            enhanced_data['RushYd'][i] = int(1400 - (rank * 40) + np.random.randint(-100, 150))
            enhanced_data['RushTD'][i] = int(12 - (rank * 0.3) + np.random.randint(-2, 3))
            enhanced_data['Rec'][i] = int(60 - (rank * 1.5) + np.random.randint(-10, 20))
            enhanced_data['RecYd'][i] = int(500 - (rank * 15) + np.random.randint(-50, 100))
            enhanced_data['RecTD'][i] = int(4 - (rank * 0.1) + np.random.randint(-1, 2))
            
        elif pos == 'WR':
            # WR stats  
            rank = i - 40  # 0-29 for WRs
            enhanced_data['Rec'][i] = int(100 - (rank * 2) + np.random.randint(-10, 15))
            enhanced_data['RecYd'][i] = int(1400 - (rank * 30) + np.random.randint(-100, 200))
            enhanced_data['RecTD'][i] = int(10 - (rank * 0.2) + np.random.randint(-2, 3))
            enhanced_data['RushYd'][i] = int(np.random.randint(0, 50)) if rank < 10 else 0
            
        elif pos == 'TE':
            # TE stats
            rank = i - 70  # 0-14 for TEs
            enhanced_data['Rec'][i] = int(80 - (rank * 3) + np.random.randint(-10, 10))
            enhanced_data['RecYd'][i] = int(900 - (rank * 40) + np.random.randint(-50, 100))
            enhanced_data['RecTD'][i] = int(8 - (rank * 0.3) + np.random.randint(-1, 2))
    
    # Add some fumbles randomly
    for i in range(num_players):
        if enhanced_data['Pos'][i] in ['RB', 'WR', 'QB']:
            enhanced_data['FumLost'][i] = int(np.random.randint(0, 3))
    
    return pd.DataFrame(enhanced_data)

def load_nfl_data():
    """Load NFL data from various sources with comprehensive player database"""
    
    print("Loading 2024 NFL Fantasy Data...")
    
    # Method 1: Try fantasydatapros GitHub (most comprehensive)
    try:
        url = "https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/2024.csv"
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            df = pd.read_csv(StringIO(response.text))
            if len(df) > 100:  # Ensure we have substantial data
                print(f"Loaded {len(df)} players from FantasyDataPros (2024)")
                return clean_and_standardize_data(df)
    except Exception as e:
        print(f"⚠️ FantasyDataPros failed: {e}")
    
    # Method 2: Try 2023 data as recent fallback
    try:
        url = "https://raw.githubusercontent.com/fantasydatapros/data/master/yearly/2023.csv"
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            df = pd.read_csv(StringIO(response.text))
            if len(df) > 100:
                print(f"Loaded {len(df)} players from 2023 season (most recent available)")
                return clean_and_standardize_data(df)
    except Exception as e:
        print(f"2023 data failed: {e}")
    
    # Method 3: Enhanced sample data with more players
    print("Using enhanced sample database (100+ players from 2024 season)")
    return create_enhanced_sample_data()

def clean_and_standardize_data(df):
    """Clean and standardize the DataFrame"""
    
    # Column mappings
    column_mapping = {
        'player': 'Player', 'player_name': 'Player', 'name': 'Player',
        'team': 'Team', 'pos': 'Pos', 'position': 'Pos',
        'passing_yards': 'PassYd', 'pass_yds': 'PassYd',
        'passing_tds': 'PassTD', 'pass_tds': 'PassTD',
        'interceptions': 'Int', 'ints': 'Int',
        'rushing_yards': 'RushYd', 'rush_yds': 'RushYd',
        'rushing_tds': 'RushTD', 'rush_tds': 'RushTD',
        'receptions': 'Rec', 'rec': 'Rec',
        'receiving_yards': 'RecYd', 'rec_yds': 'RecYd',
        'receiving_tds': 'RecTD', 'rec_tds': 'RecTD',
        'fumbles_lost': 'FumLost', 'fum_lost': 'FumLost'
    }
    
    df = df.rename(columns=column_mapping)
    
    # Ensure required columns
    required_cols = ['Player', 'Team', 'Pos', 'PassYd', 'PassTD', 'Int', 
                    'RushYd', 'RushTD', 'Rec', 'RecYd', 'RecTD', 'FumLost']
    
    for col in required_cols:
        if col not in df.columns:
            df[col] = 0
    
    # Clean data
    numeric_cols = ['PassYd', 'PassTD', 'Int', 'RushYd', 'RushTD', 'Rec', 'RecYd', 'RecTD', 'FumLost']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    df['Player'] = df['Player'].fillna('Unknown').astype(str)
    df['Team'] = df['Team'].fillna('FA').astype(str)
    df['Pos'] = df['Pos'].fillna('FLEX').astype(str).str.upper()
    
    # Standardize positions
    pos_mapping = {'DST': 'DEF', 'D/ST': 'DEF', 'DEFENSE': 'DEF'}
    df['Pos'] = df['Pos'].map(pos_mapping).fillna(df['Pos'])
    
    return df

def calculate_fantasy_points(df):
    """Calculate fantasy points and advanced metrics"""
    df = df.copy()
    df['FantasyPoints'] = 0.0
    
    # Passing: 1 pt/25 yards, 4 pts/TD, -2 pts/INT
    df['FantasyPoints'] += (df['PassYd'] / 25) + (df['PassTD'] * 4) + (df['Int'] * -2)
    
    # Rushing: 1 pt/10 yards, 6 pts/TD
    df['FantasyPoints'] += (df['RushYd'] / 10) + (df['RushTD'] * 6)
    
    # Receiving: 1 pt/rec (PPR), 1 pt/10 yards, 6 pts/TD
    df['FantasyPoints'] += df['Rec'] + (df['RecYd'] / 10) + (df['RecTD'] * 6)
    
    # Fumbles: -2 pts/fumble lost
    df['FantasyPoints'] += (df['FumLost'] * -2)
    
    df['FantasyPoints'] = df['FantasyPoints'].round(1)
    
    # Calculate Z-scores by position (position-relative value)
    df['ZScore'] = 0.0
    df['PositionRank'] = 0
    df['ValueTier'] = 'Average'
    
    for pos in df['Pos'].unique():
        pos_mask = df['Pos'] == pos
        pos_points = df.loc[pos_mask, 'FantasyPoints']
        
        if len(pos_points) > 1:
            # Calculate z-score (how many standard deviations above/below position average)
            mean_points = pos_points.mean()
            std_points = pos_points.std()
            if std_points > 0:
                df.loc[pos_mask, 'ZScore'] = ((pos_points - mean_points) / std_points).round(2)
        
        # Position ranking within position
        df.loc[pos_mask, 'PositionRank'] = pos_points.rank(method='dense', ascending=False).astype(int)
        
        # Value tiers based on z-score
        z_scores = df.loc[pos_mask, 'ZScore']
        df.loc[pos_mask & (z_scores >= 1.5), 'ValueTier'] = 'Elite'
        df.loc[pos_mask & (z_scores >= 0.5) & (z_scores < 1.5), 'ValueTier'] = 'Solid'
        df.loc[pos_mask & (z_scores >= -0.5) & (z_scores < 0.5), 'ValueTier'] = 'Average'
        df.loc[pos_mask & (z_scores < -0.5), 'ValueTier'] = 'Below Average'
    
    return df

# Load and prepare data
try:
    df_global = calculate_fantasy_points(load_nfl_data())
    print(f"Database loaded: {len(df_global)} total players")
    print(f"Positions available: {sorted(df_global['Pos'].unique())}")
except Exception as e:
    print(f"Error loading data: {e}")
    # Create minimal fallback
    df_global = create_enhanced_sample_data()
    df_global = calculate_fantasy_points(df_global)
    print("Using enhanced sample database")

# Custom CSS styling
app.layout = html.Div([
    dcc.Store(id='drafted-players-store', data=[]),
    
    # Header
    html.Div([
        html.H1("Fantasy Football Draft Dashboard", 
               style={
                   'textAlign': 'center',
                   'color': '#1f77b4',
                   'marginBottom': '20px',
                   'fontWeight': 'bold'
               }),
        html.H4("2024 Season Data • PPR Scoring • Live Draft Assistant",
               style={'textAlign': 'center', 'color': '#666', 'marginBottom': '30px'})
    ]),
    
    # Main controls row
    html.Div([
        html.Div([
            html.Label("Or Type Player Name:", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
            dcc.Input(
                id='manual-draft-input',
                type='text',
                placeholder="Enter player name...",
                style={'width': '100%', 'padding': '8px', 'fontSize': '14px'}
            )
        ], style={'width': '15%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            html.Label("Current Round:", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
            dcc.Input(
                id='round-input',
                type='number',
                value=1,
                min=1,
                max=16,
                style={'width': '100%', 'padding': '8px', 'fontSize': '16px'}
            )
        ], style={'width': '15%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            html.Label("Ranking Method:", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
            dcc.RadioItems(
                id='ranking-method',
                options=[
                    {'label': 'Fantasy Points', 'value': 'points'},
                    {'label': 'Z-Score (Position Value)', 'value': 'zscore'}
                ],
                value='zscore',
                inline=True,
                style={'fontSize': '14px'}
            )
        ], style={'width': '25%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            html.Label("Position Filter:", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
            dcc.Dropdown(
                id='position-dropdown',
                options=[
                    {'label': 'All Positions', 'value': 'ALL'},
                    {'label': 'Quarterbacks (QB)', 'value': 'QB'},
                    {'label': 'Running Backs (RB)', 'value': 'RB'},
                    {'label': 'Wide Receivers (WR)', 'value': 'WR'},
                    {'label': 'Tight Ends (TE)', 'value': 'TE'},
                    {'label': 'FLEX (RB/WR)', 'value': 'FLEX'},
                    {'label': 'Kickers (K)', 'value': 'K'},
                    {'label': 'Defense (DEF)', 'value': 'DEF'}
                ],
                value='ALL',
                style={'fontSize': '14px'}
            )
        ], style={'width': '20%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            html.Label("Draft a Player:", style={'fontWeight': 'bold', 'marginBottom': '5px'}),
            dcc.Dropdown(
                id='draft-dropdown',
                placeholder="Select player to draft...",
                style={'fontSize': '14px'}
            )
        ], style={'width': '18%', 'display': 'inline-block', 'marginRight': '2%'}),
        
        html.Div([
            html.Button("Draft Player", id='draft-btn', 
                       style={
                           'backgroundColor': '#28a745',
                           'color': 'white',
                           'border': 'none',
                           'padding': '10px 20px',
                           'fontSize': '16px',
                           'borderRadius': '5px',
                           'cursor': 'pointer',
                           'marginRight': '10px',
                           'marginTop': '24px'
                       }),
            html.Button("Clear All", id='clear-btn',
                       style={
                           'backgroundColor': '#dc3545',
                           'color': 'white',
                           'border': 'none',
                           'padding': '10px 20px',
                           'fontSize': '16px',
                           'borderRadius': '5px',
                           'cursor': 'pointer',
                           'marginTop': '24px'
                       })
        ], style={'width': '20%', 'display': 'inline-block'})
    ], style={'marginBottom': '30px'}),
    
    # Draft recommendation box
    html.Div(id='draft-recommendation', 
             style={
                 'backgroundColor': '#f8f9fa',
                 'border': '2px solid #28a745',
                 'borderRadius': '10px',
                 'padding': '20px',
                 'marginBottom': '20px',
                 'fontSize': '18px'
             }),
    
    # Stats row
    html.Div([
        html.Div([
            html.H3(id='total-players', style={'color': '#1f77b4', 'margin': '0'}),
            html.P("Total Players", style={'margin': '5px 0'})
        ], style={
            'backgroundColor': '#e3f2fd',
            'padding': '20px',
            'borderRadius': '10px',
            'textAlign': 'center',
            'width': '22%',
            'display': 'inline-block',
            'marginRight': '4%'
        }),
        
        html.Div([
            html.H3(id='drafted-count', style={'color': '#dc3545', 'margin': '0'}),
            html.P("Drafted Players", style={'margin': '5px 0'})
        ], style={
            'backgroundColor': '#ffebee',
            'padding': '20px',
            'borderRadius': '10px',
            'textAlign': 'center',
            'width': '22%',
            'display': 'inline-block',
            'marginRight': '4%'
        }),
        
        html.Div([
            html.H3(id='available-count', style={'color': '#28a745', 'margin': '0'}),
            html.P("Available Players", style={'margin': '5px 0'})
        ], style={
            'backgroundColor': '#e8f5e8',
            'padding': '20px',
            'borderRadius': '10px',
            'textAlign': 'center',
            'width': '22%',
            'display': 'inline-block',
            'marginRight': '4%'
        }),
        
        html.Div([
            html.H3(id='avg-points', style={'color': '#ff9800', 'margin': '0'}),
            html.P("Avg Fantasy Points", style={'margin': '5px 0'})
        ], style={
            'backgroundColor': '#fff3e0',
            'padding': '20px',
            'borderRadius': '10px',
            'textAlign': 'center',
            'width': '22%',
            'display': 'inline-block'
        })
    ], style={'marginBottom': '30px'}),
    
    # Main content area
    html.Div([
        # Left column - Player rankings
        html.Div([
            html.H3("Player Rankings", style={'color': '#1f77b4', 'marginBottom': '15px'}),
            html.Div(id='player-table')
        ], style={'width': '58%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        
        # Right column - Drafted players and chart
        html.Div([
            html.H3("Your Draft Board", style={'color': '#dc3545', 'marginBottom': '15px'}),
            html.Div(id='drafted-players-list', style={'marginBottom': '20px'}),
            
            html.H3("Position Analysis", style={'color': '#1f77b4', 'marginBottom': '15px'}),
            dcc.Graph(id='position-chart')
        ], style={'width': '40%', 'display': 'inline-block', 'verticalAlign': 'top', 'marginLeft': '2%'})
    ]),
    
    # League scoring info
    html.Div([
        html.Details([
            html.Summary("⚙️ League Scoring System", 
                        style={'fontSize': '18px', 'fontWeight': 'bold', 'cursor': 'pointer'}),
            html.Div([
                html.P("Passing: 1 pt/25 yards • 4 pts/TD • -2 pts/INT"),
                html.P("Rushing: 1 pt/10 yards • 6 pts/TD • -2 pts/fumble lost"),
                html.P("Receiving: 1 pt/reception (PPR) • 1 pt/10 yards • 6 pts/TD • -2 pts/fumble lost"),
                html.P("Kicking: 1-5 pts based on distance"),
                html.P("Defense: Points based on points allowed, sacks, turnovers, TDs")
            ], style={'marginTop': '10px'})
        ])
    ], style={
        'backgroundColor': '#f8f9fa',
        'padding': '15px',
        'borderRadius': '5px',
        'marginTop': '30px'
    })
], style={'padding': '20px', 'fontFamily': 'Arial, sans-serif'})

# Callbacks
@app.callback(
    [Output('draft-dropdown', 'options'),
     Output('total-players', 'children'),
     Output('drafted-count', 'children'),
     Output('available-count', 'children'),
     Output('avg-points', 'children'),
     Output('player-table', 'children'),
     Output('drafted-players-list', 'children'),
     Output('draft-recommendation', 'children'),
     Output('position-chart', 'figure')],
    [Input('position-dropdown', 'value'),
     Input('ranking-method', 'value'),
     Input('round-input', 'value'),
     Input('drafted-players-store', 'data')]
)
def update_dashboard(position_filter, ranking_method, current_round, drafted_players):
    # Filter available players
    available_df = df_global[~df_global['Player'].isin(drafted_players)].copy()
    
    # Update dropdown options
    dropdown_options = [{'label': player, 'value': player} for player in sorted(available_df['Player'].tolist())]
    
    # Stats
    total_players = len(df_global)
    drafted_count = len(drafted_players)
    available_count = len(available_df)
    avg_points = f"{available_df['FantasyPoints'].mean():.1f}" if len(available_df) > 0 else "0.0"
    
    # Get draft recommendation (always use Z-Score for recommendations)
    recommendation = get_draft_recommendation(available_df, current_round, drafted_players)
    
    # Choose ranking method
    rank_by = 'ZScore' if ranking_method == 'zscore' else 'FantasyPoints'
    
    # Filter players for display
    if position_filter == 'ALL':
        display_df = available_df.nlargest(50, rank_by)
    elif position_filter == 'FLEX':
        display_df = available_df[available_df['Pos'].isin(['RB', 'WR'])].nlargest(30, rank_by)
    else:
        display_df = available_df[available_df['Pos'] == position_filter].nlargest(30, rank_by)
    
    # Create player table
    if len(display_df) > 0:
        table_data = []
        for i, (_, player) in enumerate(display_df.iterrows(), 1):
            # Position-specific stats
            if player['Pos'] == 'QB':
                stats = f"Pass: {player['PassYd']:.0f}yd, {player['PassTD']:.0f}TD"
            elif player['Pos'] == 'RB':
                stats = f"Rush: {player['RushYd']:.0f}yd, {player['RushTD']:.0f}TD, {player['Rec']:.0f}rec"
            elif player['Pos'] in ['WR', 'TE']:
                stats = f"Rec: {player['Rec']:.0f}rec, {player['RecYd']:.0f}yd, {player['RecTD']:.0f}TD"
            else:
                stats = "Defense/Special Teams"
            
            # Value tier emoji
            tier_emoji = {'Elite': '⭐', 'Solid': '✅', 'Average': '➖', 'Below Average': '⬇️'}.get(player['ValueTier'], '➖')
            
            row_data = {
                'Rank': i,
                'Player': f"{tier_emoji} {player['Player']}",
                'Pos': player['Pos'],
                'Team': player['Team'],
                'Fantasy Pts': player['FantasyPoints'],
                'Key Stats': stats
            }
            
            if ranking_method == 'zscore':
                row_data['Z-Score'] = player['ZScore']
                row_data['Pos Rank'] = player['PositionRank']
                row_data['Tier'] = player['ValueTier']
            
            table_data.append(row_data)
        
        # Define columns based on ranking method
        if ranking_method == 'zscore':
            columns = [
                {'name': 'Rank', 'id': 'Rank', 'type': 'numeric'},
                {'name': 'Player', 'id': 'Player'},
                {'name': 'Pos', 'id': 'Pos'},
                {'name': 'Team', 'id': 'Team'},
                {'name': 'Z-Score', 'id': 'Z-Score', 'type': 'numeric', 'format': {'specifier': '.2f'}},
                {'name': 'Pos Rank', 'id': 'Pos Rank', 'type': 'numeric'},
                {'name': 'Tier', 'id': 'Tier'},
                {'name': 'Fantasy Pts', 'id': 'Fantasy Pts', 'type': 'numeric', 'format': {'specifier': '.1f'}},
                {'name': 'Key Stats', 'id': 'Key Stats'}
            ]
        else:
            columns = [
                {'name': 'Rank', 'id': 'Rank', 'type': 'numeric'},
                {'name': 'Player', 'id': 'Player'},
                {'name': 'Pos', 'id': 'Pos'},
                {'name': 'Team', 'id': 'Team'},
                {'name': 'Fantasy Pts', 'id': 'Fantasy Pts', 'type': 'numeric', 'format': {'specifier': '.1f'}},
                {'name': 'Key Stats', 'id': 'Key Stats'}
            ]
        
        player_table = dash_table.DataTable(
            data=table_data,
            columns=columns,
            style_cell={'textAlign': 'left', 'fontSize': '14px', 'padding': '10px'},
            style_header={'backgroundColor': '#1f77b4', 'color': 'white', 'fontWeight': 'bold'},
            style_data_conditional=[
                {
                    'if': {'row_index': 0},
                    'backgroundColor': '#e8f5e8',
                    'fontWeight': 'bold'
                },
                # Color code by tier
                {
                    'if': {'filter_query': '{Tier} = Elite'},
                    'backgroundColor': '#fff3e0',
                    'color': '#e65100'
                },
                {
                    'if': {'filter_query': '{Tier} = Solid'},
                    'backgroundColor': '#e8f5e8',
                    'color': '#2e7d32'
                }
            ],
            page_size=15
        )
    else:
        player_table = html.P("No players available for this position.")
    
    # Drafted players list
    if drafted_players:
        drafted_list = html.Div([
            html.Div([
                html.Span(f"{i}. {player}", style={'fontSize': '16px'}),
                html.Button("❌", id={'type': 'remove-btn', 'index': player},
                           style={
                               'backgroundColor': 'transparent',
                               'border': 'none',
                               'color': '#dc3545',
                               'cursor': 'pointer',
                               'fontSize': '16px',
                               'float': 'right'
                           })
            ], style={
                'backgroundColor': '#ffebee',
                'padding': '8px',
                'marginBottom': '5px',
                'borderRadius': '5px',
                'border': '1px solid #f44336'
            }) for i, player in enumerate(drafted_players, 1)
        ])
    else:
        drafted_list = html.P("No players drafted yet.", style={'fontStyle': 'italic', 'color': '#666'})
    
    # Position chart
    if position_filter != 'ALL' and len(display_df) >= 5:
        y_col = 'ZScore' if ranking_method == 'zscore' else 'FantasyPoints'
        y_title = 'Z-Score (Position Value)' if ranking_method == 'zscore' else 'Fantasy Points'
        
        fig = px.bar(
            display_df.head(10),
            x='Player',
            y=y_col,
            title=f'Top 10 {position_filter} Players ({y_title})',
            color=y_col,
            color_continuous_scale='viridis'
        )
        fig.update_layout(showlegend=False, height=400, xaxis_tickangle=45)
    else:
        # Position distribution pie chart
        pos_counts = available_df['Pos'].value_counts()
        fig = px.pie(
            values=pos_counts.values,
            names=pos_counts.index,
            title='Available Players by Position'
        )
        fig.update_layout(height=400)
    
    return (dropdown_options, total_players, drafted_count, available_count, avg_points,
            player_table, drafted_list, recommendation, fig)

@app.callback(
    Output('drafted-players-store', 'data'),
    [Input('draft-btn', 'n_clicks'),
     Input('clear-btn', 'n_clicks'),
     Input({'type': 'remove-btn', 'index': dash.dependencies.ALL}, 'n_clicks')],
    [State('draft-dropdown', 'value'),
     State('manual-draft-input', 'value'),
     State('drafted-players-store', 'data')],
    prevent_initial_call=True
)
def update_drafted_players(draft_clicks, clear_clicks, remove_clicks, selected_player, manual_player, current_drafted):
    ctx = dash.callback_context
    
    if not ctx.triggered:
        return current_drafted
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if 'draft-btn' in button_id:
        # Check dropdown selection first, then manual input
        player_to_draft = selected_player if selected_player else manual_player
        if player_to_draft and player_to_draft.strip() not in current_drafted:
            return current_drafted + [player_to_draft.strip()]
    elif 'clear-btn' in button_id:
        return []
    elif 'remove-btn' in button_id:
        # Extract player name from button ID
        button_data = json.loads(button_id)
        player_to_remove = button_data['index']
        return [p for p in current_drafted if p != player_to_remove]
    
    return current_drafted

@app.callback(
    [Output('draft-dropdown', 'value'),
     Output('manual-draft-input', 'value')],
    Input('draft-btn', 'n_clicks'),
    prevent_initial_call=True
)
def clear_inputs(n_clicks):
    return None, ""

if __name__ == '__main__':
    app.run(debug=True, port=8050)

Loading 2024 NFL Fantasy Data...
Using enhanced sample database (100+ players from 2024 season)
Database loaded: 105 total players
Positions available: ['DEF', 'K', 'QB', 'RB', 'TE', 'WR']
